In [1]:
import os
import pandas as pd
import numpy as np
import json
import re
import torch
from torchcrf import CRF
import random
import sys
import math
from task import DocEETask
from event_meta import EVENT_TYPES, EVENT_FIELDS, NER_LABEL2ID, NER_LABEL_LIST, EVENT_TYPE_FIELDS_PAIRS, EVENT_TYPE2ID
from model import collate_label
import torch.nn as nn
import pickle
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from pyltp import Segmentor, Postagger, Parser

In [2]:
train_edag_data = json.load(open('edag_data/sample_train.json', mode='r', encoding='utf-8'))
dev_edag_data = json.load(open('edag_data/dev.json', mode='r', encoding='utf-8'))
test_edag_data = json.load(open('edag_data/test.json', mode='r', encoding='utf-8'))
data = train_edag_data + dev_edag_data + test_edag_data

In [21]:
pos_model_path = os.path.join('ltp_model', 'pos.model')
postagger = Postagger()
postagger.load(pos_model_path)

cws_model_path = os.path.join('ltp_model', 'cws.model')
segmentor = Segmentor()
segmentor.load(cws_model_path)

parser_model_path = os.path.join('ltp_model', 'parser.model')
parser = Parser()
parser.load(parser_model_path)

In [43]:
def cal_stat(l):
    total = len(l)
    sl = sorted(l)
    t99 = sl[int(total * 0.99)]
    t95 = sl[int(total * 0.95)]
    t90 = sl[int(total * 0.9)]
    t50 = sl[int(total * 0.5)]
    return (np.mean(l), t99, t95, t90, t50)
        

len_content = []
num_sentence = []
len_sentence = []
pos_tag_count = {}
rel_count = {}
for _, ins in train_edag_data + test_edag_data:
    sentences = ins['sentences']
    for sentence in sentences:
        len_sentence.append(len(sentence))
        
        words = list(segmentor.segment(sentence))
        postags = list(postagger.postag(words))
        arcs = list(parser.parse(words, postags))
        for arc in arcs:
            rel_count[arc.relation] = rel_count.get(arc.relation, 0) + 1
            
        for postag in postags:
            if postag.startswith('n'):
                postag = 'n'
            pos_tag_count[postag] = pos_tag_count.get(postag, 0) + 1
    content = ''.join(sentences)
    len_content.append(len(content))
    num_sentence.append(len(sentences))
    

print(cal_stat(len_content))
print(cal_stat(num_sentence))
print(cal_stat(len_sentence))

(950.2193197755961, 3435, 2163, 1602, 767)
(20.228611500701263, 59, 43, 34, 17)
(46.97402586147126, 210, 137, 110, 33)


In [47]:
words

['2018年', '12月', '28日']

In [59]:
arc_head = len(''.join(words[:arcs[1].head - 1]))
arc_head

8

In [45]:
l = list(rel_count.items())
l.sort(key=lambda x: x[1], reverse=True)
s = sum(list(map(lambda x: x[1], l)))
print(len(l))
base = 0
for postag, num in l:
    base += num / s
    print(postag, num, num / s, base)

14
ATT 1070907 0.37659128911957546 0.37659128911957546
WP 410325 0.14429340802515048 0.520884697144726
VOB 262449 0.09229186777016442 0.6131765649148904
COO 255984 0.09001840921199078 0.7031949741268811
ADV 250084 0.08794363651388955 0.7911386106407707
SBV 189564 0.06666139182082403 0.8578000024615948
RAD 121049 0.042567654293636605 0.9003676567552314
HED 115384 0.04057552084706991 0.9409431776023014
POB 97041 0.034125087694312134 0.9750682652966135
LAD 49748 0.017494202065277974 0.9925624673618915
CMP 11152 0.0039216720558008355 0.9964841394176923
FOB 9174 0.0032260957173526604 0.999710235135045
DBL 812 0.00028554498828105084 0.999995780123326
IOB 12 4.219876674104199e-06 1.0000000000000002


In [20]:
l = list(pos_tag_count.items())
l.sort(key=lambda x: x[1], reverse=True)
s = sum(list(map(lambda x: x[1], l)))
print(len(l))
base = 0
for postag, num in l:
    base += num / s
    print(postag, num, num / s, base)

18
n 1064021 0.3741697832214187 0.3741697832214187
v 616005 0.21662209421929643 0.5907918774407152
wp 410316 0.14429024311764488 0.73508212055836
m 148561 0.0522424248817995 0.7873245454401595
u 127627 0.04488085002382472 0.8322053954639842
p 100124 0.035209244343167405 0.8674146398071516
a 72898 0.025635047482403993 0.8930496872895556
r 70988 0.02496338377844241 0.918013071067998
q 60810 0.02138422504602303 0.939397296114021
c 58609 0.02061022933271442 0.9600075254467354
d 57654 0.020274397480733626 0.9802819229274691
b 39474 0.013881284319465764 0.9941632072469349
j 10858 0.003818285077285283 0.9979814923242202
ws 3866 0.0013595036018405695 0.9993409959260607
i 1801 0.0006333331575051386 0.9999743290835659
h 37 1.3011286411821282e-05 0.9999873403699777
z 27 9.494722516734448e-06 0.9999968350924945
k 9 3.1649075055781495e-06 1.0


In [15]:
s

2843685

In [17]:
contents = []
events_count = []
for _, ins in data:
    events = ins['recguid_eventname_eventdict_list']
    content = ''.join(ins['sentences'])
    contents.append(len(content))
    if len(events) > 1:
        events_count.append(1)
contents = np.array(contents)

In [20]:
len(dev_edag_data)

3204

In [21]:
default_task_config = {
    'test_file': 'ccks 4_2 Data/event_element_dev_data.txt',
    'train_file': 'ccks 4_2 Data/event_element_train_data_label.txt',
    'output_dir': 'output',
    'model_save_dir': 'save_model',
    'eval_save_dir': 'save_eval',
    'test_save_dir': 'save_test',
    'eval_json_file': 'eval-%d.json',
    'eval_obj_file': 'eval-obj-%d.pkl',
    'save_test_file': 'test-%d.txt',
    'model_file': '%s-%d.cpt',
    'random_seed': 666,

    'epoch': 300,
    'train_doc_batch_size': 2,
    'eval_doc_batch_size': 2,
    'test_doc_batch_size': 2,
    'accum_batch_size': 64,
    'not_accum_optim_epoch': 1,

    'use_schedule_sampling': True,
    'min_teacher_prob': 0.1,
    'schedule_epoch_start': 1,
    'schedule_epoch_length': 10,# the number of epochs to linearly transit to the min_teacher_prob

    'skip_train': False,
    'skip_eval': False,
    'save_model': True,
    'resume_model': True,

    'max_tokens_length': 500,
    'max_sent_num': 5,
    'sent_batch_size': 5,

    'dev_ratio': 0.05,
    'text_norm': True,
    
    'use_bert': False,
    'bert_model_name': 'hfl/rbt3',
    'bert_dir': 'rbt3',
    'num_bert_layer': None,
    'bert_add_cls_sep': False,

    'use_xlnet': True,
    'xlnet_doc_bidirection': False,
    'xlnet_doc_reverse': False,
    'xlnet_dir': 'xlnet_chinese',
    'num_xlnet_layer': 4,
    'xlnet_mem_len': 1024,

    'hidden_size': 768,
    'dropout': 0.1,

    'ee_method': 'EDAG', # [GreedyDec, EDAG]
    'use_edag_graph': False, # useless bullshit
    'trainable_pos_emb': False,
    'span_drange_fuse': 'add', # ['add', 'concat']
    'ner_label_count_limit': None,
    'ner_label_sentence_length': 500,

    'cuda': True,
    'use_crf': False,
    'use_token_role': True,
    'use_pos_emb': False,
    'use_doc_enc': False, # consider
    'use_rnn_enc': None,
    'rnn_bidirection': False,

    'num_tf_layer': 4,
    'ff_size': 1024,
    'neg_field_loss_scaling': 1.0, #prefer FN over FP
    'pooling': 'max', # ['max', 'mean', 'AWA']
    'learning_rate': 1e-4,

    'EVENT_TYPES': EVENT_TYPES,
    'EVENT_FIELDS': EVENT_FIELDS,
    'NER_LABEL_LIST': NER_LABEL_LIST,
    'NER_LABEL2ID': NER_LABEL2ID,
    'EVENT_TYPE2ID': EVENT_TYPE2ID,
    'EVENT_TYPE_FIELDS_PAIRS': EVENT_TYPE_FIELDS_PAIRS,

    'debug_data_num': None,
    'debug_data_id_test': None,
    'ltp_path': 'ltp_model',
    'cut_word_task': False,
    'validate_doc_file': 'validate_doc.pkl',
    'test_doc_file': 'test_doc.pkl'
}
task = DocEETask(default_task_config)
task.load_data()
task.preprocess()

  0%|          | 18/3956 [00:00<00:22, 176.89it/s]

load data finsih


100%|██████████| 740/740 [00:06<00:00, 105.86it/s][A

preprocess finish



100%|██████████| 3956/3956 [00:51<00:00, 83.09it/s]

In [8]:
count = 0
for ins in task.train:
    for event in ins['events']:
        if '死亡年龄' in event:
            count += 1

In [9]:
count

29

In [5]:
eval_obj = pickle.load(open('output/save_eval/eval-obj-31.pkl', mode='rb'))


100%|██████████| 740/740 [00:26<00:00, 85.19it/s]

In [7]:
count = []
decode_idx = []
for idx, ins in enumerate(eval_obj['ins']):
    if len(ins['merge_sentences']) > 3:
        count.append(ins)
        decode_idx.append(idx)
        
for ins in task.train:
    if ins['doc_id'] == '2665361':
        break

chh = 2
beg_set = set()
end_set = set()
ins = count[chh]#task.train[0]
content = ins['content']
n_content = content
for event in ins['events']:
    label_dict = {}
    f_label_dict = {}
    for role_type, span in event.items():
        if role_type == 'event_type' or role_type == 'event_id' or not span:
            continue
        label_dict[role_type] = []
        find_idx = -0.5
        find_idx_set = []
        while find_idx != -1:
            find_idx = content.find(span, int(find_idx + 1))
            if find_idx != -1:
                assert content[find_idx: find_idx + len(span)] == span
                if find_idx - 1 >= 0 and content[find_idx - 1] == '】':
                    continue
                label_dict[role_type].append((find_idx, find_idx + len(span)))
#                 content = content[:find_idx] + '【BEG】' + span + '【END】' + content[find_idx + len(span):]
#                 find_idx += len('【SEP】')
    #print(label_dict)
    event_pos = []
    for role_type, dranges in label_dict.items():
        #print(role_type)
        role_pos = np.array(list(map(lambda drange: drange[0], dranges))).mean()
        #print(role_pos)
        event_pos.append(role_pos)
    event_pos = sum(event_pos) / len(event_pos)
    #print(event_pos)
    for role_type, dranges in label_dict.items():
        ch = float('inf')
        ch_drange = None
        for drange in dranges:
            if abs(drange[0] - event_pos) < ch:
                ch = abs(drange[0] - event_pos)
                ch_drange = drange
        f_label_dict[role_type] = ch_drange
    #print(f_label_dict)
    nn_content = ''
    for _, drange in f_label_dict.items():
        beg_set.add(drange[0])
        end_set.add(drange[1])
for idx, c in enumerate(n_content):
    if idx in beg_set:
        nn_content += '【BEG】'
    if idx in end_set:
        nn_content += '【END】'
    nn_content += c
    
    #break
print(nn_content) 
print('----------------')
print(content)
#print(eval_obj['decode_res'][decode_idx[chh]])
ins['events']

多喜爱：关于控股股东所持公司股票遭遇平仓导致被动减持累计达到1%暨风险提示的公告<br>证券代码：002761 证券简称：多喜爱 公告编号：2019-009<br>多喜爱集团股份有限公司<br>关于控股股东所持公司股票遭遇平仓导致被动减持累计达到 1%<br>暨风险提示的公告<br>本公司及董事会全体成员保证信息披露内容的真实、准确、完整，没有虚假记载、误导性陈述或重大遗漏。<br>多喜爱集团股份有限公司（以下简称“公司”）于 2019年2月11日收到公司控股股东、实际控制人陈军先生与黄娅妮女士函告，获悉其质押给第一创业证券股份有限公司（以下简称<br>“第一创业”）的部分公司股票因触发协议约定的违约条款遭遇强制平仓导致被动减持。现将<br>有关情况公告如下：<br>一、股东被动减持情况<br>1、本次被动减持股份情况<br>股东名称 被动减持时间 减持方式 被动减持股数（股）占公司总股本比例<br>陈军 2019年2月1日集中竞价交易<br>【BEG】899800【END】 0.44%<br>黄娅妮 【BEG】2019年2月1日【END】集中竞价交易 【BEG】283515【END】 0.14%<br>合计 - 1183315 0.58%<br>2、本次被动减持前后持股情况股东名称<br>本次被动减持前持有公司份 本次被动减持后持有公司股份<br>持股数（股） 占总股本比例 持股数（股） 占总股本比例<br>陈军 58162032 28.51% 57262232 28.07%<br>【BEG】黄娅妮【END】 44711854 21.92% 44428339 21.78%<br>二、其他情况说明1、公司于 2019年1月9日在指定信息披露媒体披露了《关于控股股东、实际控制人存在可能被动减持公司股票风险的预披露公告》（公告编号：2019-001），【BEG】陈军【END】先生、黄娅妮女士由于在第一创业办理的部分股票质押式回购交易触发协议约定的违约条款，第一创业有权对质押的标的证券进行违约处置。同时，第一创业自 2019年1月9日起 15 个交易日（针对集中竞价交易方式）或 3 个交易日（针对大宗交易方式）后的 6 个月内，通过集中竞价或大宗交易方式减持公司股份，且在减持期间内任意连续 90 个自然日内通过集中竞价交易减持股份的总数不超过公司总股本的 1%，通过大宗交

[{'event_id': '4405061',
  '减持的股东': '陈军',
  '减持金额': '899800',
  'event_type': '股东减持',
  '减持开始日期': '2019年2月1日'},
 {'event_id': '4108135',
  '减持的股东': '黄娅妮',
  '减持金额': '283515',
  'event_type': '股东减持',
  '减持开始日期': '2019年2月1日'}]

In [2]:
dev_file = open('ccks 4_2 Data/event_element_dev_data.txt', mode='r', encoding='utf-8')
train_file = open('ccks 4_2 Data/event_element_train_data_label.txt', mode='r', encoding='utf-8')

In [3]:
train = []
dev = []
for line in train_file.readlines():
    train.append(json.loads(line))
for line in dev_file.readlines():
    dev.append(json.loads(line))

In [23]:
task.test[0]

{'content': '香港交易及結算所有限公司及香港聯合交易所有限公司對本公告的內容概不負責，對其準確性或完整性亦不發表任何聲明，並明確表示，概不對因本公告全部或任何部份內容而產生或因依賴該等內容而引致的任何損失承擔任何責任。PROVIEW&nbsp;INTERNATIONAL&nbsp;HOLDINGS&nbsp;LIMITED唯冠國際控股有限公司*(於百慕達註冊成立之有限公司)(股份代號：334)深圳唯冠破產案件最新進展公告茲提述唯冠國際控股有限公司(「本公司」)於二零一二年三月七日以及二零一二年六月十五日作出的關於本公司子公司唯冠科技(深圳)有限公司(「深圳唯冠」)所涉及之破產訴訟的相關公告。董事會謹此宣佈，廣東省高級人民法院於二零一二年九月十日就富邦產物保險股份有限公（「富邦」）申請深圳唯冠破產清算一案作出終審裁定，該裁定如下:一．撤銷廣東省深圳市中級人民法院（「深圳中院」）於二零一二年三月二十七日作出的不予&apos;受理富邦申請深圳唯冠破產清算的裁定；二．指令深圳中院受理富邦申請深圳唯冠破產清算一案。若有關深圳唯冠破產清算案有任何進一步的發展，本公司將會於適當時候另行刊發相關之公告。應本公司之要求，股份已於二零一零年八月二日下午二時三十分起暫停買賣，直至另行通告為止。承董事會命唯冠國際控股有限公司孫敏主席香港，二零一二年九月二十六日於本公告日期，本公司之董事會成員為魯桂芳先生、張宜巽先生、韓蘇先生、余根明先生及孫敏女士。*僅供識別',
 'doc_id': '2485307',
 'content_norm': '香港交易及结算所有限公司及香港联合交易所有限公司对本公告的内容概不负责,对其准确性或完整性亦不发表任何声明,并明确表示,概不对因本公告全部或任何部份内容而产生或因依赖该等内容而引致的任何损失承担任何责任。proview&nbsp;international&nbsp;holdings&nbsp;limited唯冠国际控股有限公司*(于百慕达注册成立之有限公司)(股份代号:334)深圳唯冠破产案件最新进展公告兹提述唯冠国际控股有限公司(「本公司」)于二零一二年三月七日以及二零一二年六月十五日作出的关于本公司子公司唯冠科技(深圳)有限公司(「深圳唯冠」)所涉及之破产诉讼的相关公告。董事会谨此宣布,广东省高级人民法院于二零一二年九月十日就富邦产物

In [18]:
res = {}
for i, ins in enumerate(task.train + dev):
    if re.search('简称|下称', ins['content']) is None:
        print('bad')
        break

bad


In [19]:
ins

{'content': '香港交易及結算所有限公司及香港聯合交易所有限公司對本公告之內容概不負責，對其準確性或完整性亦不發表任何聲明，並明確表示概不就因本公告全部或任何部份內容而產生或因倚賴該等內容而引致之任何損失承擔任何責任。CHINA&nbsp;OCEAN&nbsp;INDUSTRY&nbsp;GROUP&nbsp;LIMITED中海重工集團有限公司（於百慕達註冊成立之有限公司）（股份代號：00651）內幕消息針對本公司一間附屬公司之破產公告本公告乃由中海重工集團有限公司（「本公司」，連同其附屬公司統稱「本集團」）根據香港法例第571章證券及期貨條例第XIVA部項下之內幕消息條文及香港聯合交易所有限公司（「聯交所」）證券上市規則（「上市規則」）第13.09(2)(a)條及第13.25(1)(e)條而作出。授出破產令茲提述本公司日期為二零一九年七月十九日之公告，內容有關針對本公司一間附屬公司南通華凱重工有限公司（「南通華凱」）之破產程序。本公司董事會（「董事會」）宣佈，於二零一九年十二月二十日，中華人民共和國（「中國」）江蘇省如皋市人民法院（「法院」）裁定（其中包括）南通華凱破產。於二零一九年十二月二十六日，法院裁定，基於破產管理人申請，南通華凱之附屬公司與南通華凱合併破產（統稱「破產」）。茲提述本公司日期為二零一九年六月十三日、二零一九年七月十二日、二零一九年七月十五日、二零一九年七月十九日及二零一九年九月三十日之公告，內容有關出售南通華凱之60%股權（「出售事項」）。鑒於破產，本公司預期購股協議將於二零一九年十二月三十一日失效。因此，本公司將不再繼續進行出售事項。本公司正在就破產之影響及使相關程序生效而將採取之相關措施尋求本公司中國法律顧問及核數師之意見。本公司將就與之有關之任何進一步資料或有關破產之任何重大發展刊發進一步公告。本公司股東及潛在投資者在買賣本公司證券時務請審慎行事。承董事會命中海重工集團有限公司主席李明香港，二零一九年十二月二十七日於本公告日期，本公司董事會由兩名執行董事李明先生及張士宏先生；以及三名獨立非執行董事項思英女士、胡柏和先生及向穎女士組成。',
 'doc_id': '2197355',
 'events': [{'受理法院': '江蘇省如皋市人民法院',
   'event_type': '破产清算',
   'event_id'

In [ ]:
for i, ins in enumerate(train):
    
    for event in ins['events']:
        event_type = event['event_type']
        
        for event_role, span in event.items():
            if event_role == 'event_id' or event_role == 'event_type' or not span:
                continue
            
    
    nodes = list(nodes)
    edegs = set()
    for event in ins['events']:
        event_type = event['event_type']
        for event_role, span in event.items():
            if event_role == 'event_id' or event_role == 'event_type' or not span:
                continue
            a = nodes.index(span)
            for _event_role, _span in event.items():
                if _event_role == 'event_id' or _event_role == 'event_type' or not _span:
                    continue
                b = nodes.index(_span)
                edegs.add((a, b))
                edegs.add((b, a))
    num_nodes.append(len(nodes))
    num_edegs.append(len(edegs))
    
    if len(edegs) / (len(nodes) * len(nodes)) <= 0.7:
        mult_events.append(ins['events'])
        mult_event_content.append(ins)
num_nodes = np.array(num_nodes)
num_edegs = np.array(num_edegs)

In [80]:
EVENT_TYPES = ['破产清算', '重大安全事故', '股东减持', '股权质押', '股东增持', '股权冻结', '高层死亡', '重大资产损失', '重大对外赔付']
num_nodes = []
num_edegs = []
mult_events = []
mult_event_content = []
for i, ins in enumerate(train):
    nodes = set()
    for event in ins['events']:
        event_type = event['event_type']
        for event_role, span in event.items():
            if event_role == 'event_id' or event_role == 'event_type' or not span:
                continue
            nodes.add(span)
    
    nodes = list(nodes)
    edegs = set()
    for event in ins['events']:
        event_type = event['event_type']
        for event_role, span in event.items():
            if event_role == 'event_id' or event_role == 'event_type' or not span:
                continue
            a = nodes.index(span)
            for _event_role, _span in event.items():
                if _event_role == 'event_id' or _event_role == 'event_type' or not _span:
                    continue
                b = nodes.index(_span)
                edegs.add((a, b))
                edegs.add((b, a))
    num_nodes.append(len(nodes))
    num_edegs.append(len(edegs))
    
    if len(edegs) / (len(nodes) * len(nodes)) <= 0.7:
        mult_events.append(ins['events'])
        mult_event_content.append(ins)
num_nodes = np.array(num_nodes)
num_edegs = np.array(num_edegs)

mult_event_count = {}
for events in mult_events:
    for event in events:
        event_type = event['event_type']
        mult_event_count[event_type] = mult_event_count.get(event_type, 0) + 1

In [82]:
ch = 50
print(mult_event_content[ch]['content'], mult_event_content[ch]['doc_id'])
mult_events[ch]

英飞特：关于持股5%以上股东股份减持计划期限届满以及未来股份减持计划预披露的公告<br>证券代码：300582 证券简称：英飞特 公告编号：2019-015<br>英飞特电子（杭州）股份有限公司<br>关于持股5%以上股东股份减持计划期限届满以及未来股份减持计划<br>预披露的公告公司股东浙江华睿泰信创业投资有限公司、浙江尚志股权投资合伙企业(有限合伙)保证向公司提供的信息内容真实、准确、完整，没有虚假记载、误导性陈述或重大遗漏。<br>本公司及董事会全体成员保证公告内容与信息披露义务人提供的信息一致。<br>特别提示：<br>1、持本公司股份13360000（占公司总股本的6.62%）的股东浙江华睿泰<br>信创业投资有限公司计划自本公告发布之日起15个交易日后的六个月内，通过集中竞价交易或大宗交易等方式合计减持公司股份不超过8905000股（不超过公司总股本的4.41%）。若此期间公司有送股、资本公积转增股本等股本变动事项，应对该数量进行相应的调整。<br>2、持本公司股份13199514股（占公司总股本的6.54%）的股东浙江尚志股<br>权投资合伙企业（有限合伙）计划自本公告发布之日起15个交易日后的六个月内，通过集中竞价交易或大宗交易等方式合计减持公司股份不超过8119121股（不超过公司总股本的4.02%）。若此期间公司有送股、资本公积转增股本等股本变动事项，应对该数量进行相应的调整。<br>一、股东减持计划期限届满及实施情况<br>英飞特电子（杭州）股份有限公司（以下简称 “公司”）于2018年8月7日在中国证监会指定的创业板信息披露网站发布了《关于持股5%以上股东股份减持计划期限届满以及未来股份减持计划预披露的公告》（公告编号：2018-094），公司股东浙江华睿泰信创业投资有限公司（以下简称 “华睿泰信”）计划在该公告<br>发布之日起15个交易日后的六个月内，通过集中竞价交易或大宗交易等方式合计减持公司股份不超过5090000股，即不超过当时公司总股本的2.53%。公司于2018年11月28日在中国证监会指定的创业板信息披露网站发布了《关于持股5%以上股东股份减持计划实施进展的公告》（公告编号：2018-144），华睿泰信本次减持计划期间过半。<br>公司于2018年8月7日在中国证监会指定的创业板信息披露网站发布了《关于<br>持股5%以上股东

[{'event_id': '4325358',
  '减持的股东': '浙江华睿泰信创业投资有限公司',
  '减持金额': '100000',
  'event_type': '股东减持',
  '减持开始日期': '2019年1月28日'},
 {'event_id': '4945385',
  '减持的股东': '浙江尚志股权投资合伙企业(有限合伙)',
  '减持金额': '100000',
  'event_type': '股东减持',
  '减持开始日期': '2018年9月13日'},
 {'event_id': '4521341',
  '减持的股东': '浙江尚志股权投资合伙企业(有限合伙)',
  '减持金额': '518700',
  'event_type': '股东减持',
  '减持开始日期': '2019年1月22日'},
 {'event_id': '4263682',
  '减持的股东': '浙江华睿泰信创业投资有限公司',
  '减持金额': '210000',
  'event_type': '股东减持',
  '减持开始日期': '2019年1月22日'},
 {'event_id': '4773498',
  '减持的股东': '浙江尚志股权投资合伙企业(有限合伙)',
  '减持金额': '100000',
  'event_type': '股东减持',
  '减持开始日期': '2018年9月14日'}]

In [55]:
mult_event_count

{'破产清算': 1015,
 '重大安全事故': 391,
 '股东减持': 1151,
 '股权质押': 563,
 '股东增持': 843,
 '股权冻结': 710,
 '高层死亡': 324,
 '重大资产损失': 397,
 '重大对外赔付': 127}

In [11]:
EVENT_TYPES = ['破产清算', '重大安全事故', '股东减持', '股权质押', '股东增持', '股权冻结', '高层死亡', '重大资产损失', '重大对外赔付']
event_field_count = [{} for _ in EVENT_TYPES]
for i, ins in enumerate(train):
    for event in ins['events']:
        event_type = event['event_type']
        event_idx = EVENT_TYPES.index(event_type)
        field_count = event_field_count[event_idx]
        for event_role, span in event.items():
            if event_role == 'event_id' or event_role == 'event_type' or not span:
                continue
            field_count[event_role] = field_count.get(event_role, 0) + 1

for i, event_type in enumerate(EVENT_TYPES):
    print(event_type)
    pair = list(event_field_count[i].items())
    pair = sorted(pair, key=lambda x: x[1], reverse=True)
    print(pair)
    print(list(map(lambda x: x[0], pair)))
    print('-------------')

破产清算
[('公司名称', 1010), ('公告时间', 976), ('受理法院', 864), ('裁定时间', 571), ('公司行业', 51)]
['公司名称', '公告时间', '受理法院', '裁定时间', '公司行业']
-------------
重大安全事故
[('公司名称', 390), ('公告时间', 380), ('伤亡人数', 147), ('损失金额', 84), ('其他影响', 36)]
['公司名称', '公告时间', '伤亡人数', '损失金额', '其他影响']
-------------
股东减持
[('减持的股东', 1151), ('减持金额', 1151), ('减持开始日期', 1120)]
['减持的股东', '减持金额', '减持开始日期']
-------------
股权质押
[('质押金额', 563), ('质押开始日期', 532), ('接收方', 524), ('质押方', 497), ('质押结束日期', 249)]
['质押金额', '质押开始日期', '接收方', '质押方', '质押结束日期']
-------------
股东增持
[('增持金额', 843), ('增持的股东', 843), ('增持开始日期', 822)]
['增持金额', '增持的股东', '增持开始日期']
-------------
股权冻结
[('被冻结股东', 710), ('冻结金额', 710), ('冻结开始日期', 693), ('冻结结束日期', 272)]
['被冻结股东', '冻结金额', '冻结开始日期', '冻结结束日期']
-------------
高层死亡
[('公司名称', 324), ('高层人员', 323), ('高层职务', 318), ('死亡/失联时间', 187), ('死亡年龄', 29)]
['公司名称', '高层人员', '高层职务', '死亡/失联时间', '死亡年龄']
-------------
重大资产损失
[('公司名称', 395), ('公告时间', 346), ('损失金额', 209), ('其他损失', 22)]
['公司名称', '公告时间', '损失金额', '其他损失']
-------------
重大对外赔付
[('公告时间'

In [11]:
doc_span_num = []
for i, ins in enumerate(train):
    span_num = 0
    for event in ins['events']:
        event_type = event['event_type']
        for event_role, span in event.items():
            if event_role == 'event_id' or event_role == 'event_type' or not span:
                continue
            span_num += 1
    doc_span_num.append(span_num)
doc_span_num = np.array(doc_span_num)
print(sum(doc_span_num) / len(doc_span_num))
print(min(doc_span_num), max(doc_span_num))
for bound in range(1, 21):
    print(bound, sum(doc_span_num < bound) / len(doc_span_num))

4.493174924165824
1 20
1 0.0
2 0.014155712841253791
3 0.14004044489383216
4 0.49443882709807885
5 0.7709807886754297
6 0.8187563195146613
7 0.8710819009100101
8 0.8756319514661274
9 0.8935793731041456
10 0.9223963599595552
11 0.9287158746208292
12 0.9320020222446916
13 0.9529828109201214
14 0.9542467138523761
15 0.9575328614762386
16 0.9941860465116279
17 0.9957027300303337
18 0.9959555106167847
19 0.9967138523761375
20 0.9977249747219413


In [28]:
len_arr = []
for i, ins in enumerate(train):
    len_arr.append(len(ins['content']))
print(sum(len_arr) / len(len_arr))
len_arr = np.array(len_arr)
for bound in np.arange(1000, 4000, 100):
    print(bound, sum(len_arr < bound) / len(len_arr))

1253.324570273003
1000 0.5700202224469161
1100 0.6134984833164813
1200 0.6531850353892821
1300 0.6885743174924166
1400 0.7323053589484327
1500 0.7601112234580384
1600 0.7917087967644085
1700 0.8217896865520729
1800 0.8430232558139535
1900 0.8650151668351871
2000 0.8816986855409504
2100 0.8963599595551062
2200 0.9092517694641051
2300 0.916582406471183
2400 0.9271991911021233
2500 0.9370576339737108
2600 0.9448938321536906
2700 0.9507077856420627
2800 0.9542467138523761
2900 0.9577856420626896
3000 0.9600606673407482
3100 0.9641051567239636
3200 0.9658746208291203
3300 0.967896865520728
3400 0.9696663296258847
3500 0.9716885743174924
3600 0.9729524772497472
3700 0.974469160768453
3800 0.9757330637007078
3900 0.9764914054600606


In [12]:
train[1904]['events'][0]['增持的股东'] = '微医集团（浙江）有限公司'
train[3693]['events'][-1]['被冻结股东'] = '上海九川投资（集团）有限公司'
train[3693]['content'] = train[3693]['content'].replace('上海九川投资(集团)有限公司', '上海九川投资（集团）有限公司')


count = {}
for i, ins in enumerate(train):
    l = len(ins['events'])
    arr = count.get(l)
    if arr is not None:
        arr.append(ins)
    else:
        count[l] = [ins]
    
#     if re.search(r'[；]', ins['content']) is not None:
#         print(ins['content'])
#         break
                
print(len(train))
print(list(count.keys()))
for i in range(1, 6):
    print('-------')
    arr = count.get(i)
    print(len(arr))
    count2 = {}
    len_arr = []
    for ins in arr:
        len_arr.append(len(ins['content']))
        for event in ins['events']:
            event_type = event['event_type']
            count2[event_type] = count2.get(event_type, 0) + 1
#             for event_role, span in event.items():
#                 if event_role == 'event_type' or event_role == 'event_id':
#                     continue
    len_arr = sorted(len_arr)
    print(len_arr[0], len_arr[-1])
    print(len_arr[len(len_arr) // 2])
    print(sum(len_arr) / len(len_arr))
    for ee_type, _count in count2.items():
        print(ee_type, _count)

3956
[1, 5, 4, 2, 3]
-------
3230
171 43249
782
1171.790092879257
破产清算 1015
重大安全事故 391
股东减持 151
股权质押 318
股东增持 220
股权冻结 287
高层死亡 324
重大资产损失 397
重大对外赔付 127
-------
323
430 3699
1286
1385.3498452012384
股东减持 168
股权质押 158
股东增持 166
股权冻结 154
-------
135
512 4854
1534
1600.111111111111
股东减持 147
股权质押 27
股东增持 150
股权冻结 81
-------
100
792 3785
1577
1694.16
股东减持 180
股权质押 20
股东增持 112
股权冻结 88
-------
168
822 5519
1869
2026.017857142857
股东减持 505
股权质押 40
股东增持 195
股权冻结 100
